# Chapter 33: Analytics & Monitoring

Understanding user behavior, application performance, and error patterns is crucial for maintaining a production-grade Next.js application. Modern analytics requires balancing comprehensive data collection with privacy regulations like GDPR and CCPA, while real-time monitoring helps identify issues before they impact users. The App Router's hybrid rendering model requires specific instrumentation strategies to capture metrics across server components, client components, and edge functions.

By the end of this chapter, you'll master implementing privacy-compliant analytics solutions, tracking Core Web Vitals in real-world conditions, setting up comprehensive error tracking and alerting, building custom event tracking pipelines, monitoring server-side performance metrics, and creating dashboards for operational visibility.

## 33.1 Privacy-First Analytics

Implement analytics solutions that respect user privacy while providing actionable insights.

### Vercel Analytics and Speed Insights

```typescript
// app/layout.tsx
import { Analytics } from '@vercel/analytics/react';
import { SpeedInsights } from '@vercel/speed-insights/next';
import Script from 'next/script';

export default function RootLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <html lang="en">
      <body>
        {children}
        
        {/* Vercel Analytics - privacy-friendly, no cookies */}
        <Analytics 
          mode={process.env.NODE_ENV === 'development' ? 'development' : 'production'}
          debug={process.env.NODE_ENV === 'development'}
        />
        
        {/* Speed Insights for Core Web Vitals */}
        <SpeedInsights 
          sampleRate={0.8} // Track 80% of sessions
        />
      </body>
    </html>
  );
}
```

### Custom Privacy-Compliant Tracking

```typescript
// lib/analytics/provider.ts
type AnalyticsEvent = {
  name: string;
  properties?: Record<string, any>;
  timestamp: number;
  sessionId: string;
  path: string;
};

class AnalyticsProvider {
  private endpoint: string;
  private queue: AnalyticsEvent[] = [];
  private flushInterval: number;
  private sessionId: string;

  constructor(endpoint: string, flushInterval = 5000) {
    this.endpoint = endpoint;
    this.flushInterval = flushInterval;
    this.sessionId = this.generateSessionId();
    
    // Flush queue periodically
    if (typeof window !== 'undefined') {
      setInterval(() => this.flush(), this.flushInterval);
      
      // Flush on page unload
      window.addEventListener('beforeunload', () => this.flush());
    }
  }

  private generateSessionId(): string {
    // Generate privacy-friendly session ID (no personal data)
    return `${Date.now()}-${Math.random().toString(36).substr(2, 9)}`;
  }

  track(eventName: string, properties?: Record<string, any>) {
    // Respect Do Not Track
    if (typeof navigator !== 'undefined' && navigator.doNotTrack === '1') {
      return;
    }

    // Check for consent (GDPR/CCPA)
    if (!this.hasConsent()) {
      return;
    }

    const event: AnalyticsEvent = {
      name: eventName,
      properties,
      timestamp: Date.now(),
      sessionId: this.sessionId,
      path: typeof window !== 'undefined' ? window.location.pathname : '',
    };

    this.queue.push(event);

    // Flush immediately for critical events
    if (eventName === 'purchase' || eventName === 'signup') {
      this.flush();
    }
  }

  private hasConsent(): boolean {
    // Check localStorage for consent
    if (typeof window === 'undefined') return true;
    const consent = localStorage.getItem('analytics-consent');
    return consent === 'granted';
  }

  private async flush() {
    if (this.queue.length === 0) return;

    const events = [...this.queue];
    this.queue = [];

    try {
      // Send to your analytics endpoint
      await fetch(this.endpoint, {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ events }),
        // Use keepalive for unload events
        keepalive: true,
      });
    } catch (error) {
      // Re-queue on failure
      this.queue.unshift(...events);
    }
  }
}

export const analytics = new AnalyticsProvider('/api/analytics');
```

### Server-Side Analytics

```typescript
// app/api/analytics/route.ts
import { NextRequest } from 'next/server';
import { kv } from '@vercel/kv';

export async function POST(req: NextRequest) {
  const { events } = await req.json();
  
  // Batch process events
  const pipeline = kv.pipeline();
  
  for (const event of events) {
    // Store in time-series format
    const date = new Date(event.timestamp).toISOString().split('T')[0];
    const key = `analytics:${date}:${event.name}`;
    
    // Increment counters
    pipeline.hincrby(key, 'count', 1);
    
    // Store unique sessions (HyperLogLog for cardinality)
    pipeline.pfadd(`analytics:sessions:${date}`, event.sessionId);
    
    // Store path tracking
    if (event.path) {
      pipeline.hincrby(`analytics:paths:${date}`, event.path, 1);
    }
  }
  
  await pipeline.exec();
  
  return Response.json({ success: true });
}

// Aggregate daily stats
export async function GET(req: NextRequest) {
  const { searchParams } = new URL(req.url);
  const date = searchParams.get('date') || new Date().toISOString().split('T')[0];
  
  const stats = await kv.hgetall(`analytics:${date}`);
  const sessions = await kv.pfcount(`analytics:sessions:${date}`);
  
  return Response.json({
    date,
    events: stats,
    uniqueSessions: sessions,
  });
}
```

## 33.2 Core Web Vitals Monitoring

Track performance metrics from real users in production environments.

### Web Vitals Tracking

```typescript
// lib/vitals.ts
import { Metric, onCLS, onFCP, onFID, onINP, onLCP, onTTFB } from 'web-vitals';
import { analytics } from './analytics/provider';

const vitalsUrl = '/api/vitals';

function getConnectionSpeed(): string {
  return (navigator as any).connection?.effectiveType || 'unknown';
}

function sendToAnalytics(metric: Metric) {
  const body = JSON.stringify({
    name: metric.name,
    value: metric.value,
    rating: metric.rating, // 'good' | 'needs-improvement' | 'poor'
    delta: metric.delta,
    navigationType: metric.navigationType,
    id: metric.id,
    page: window.location.pathname,
    speed: getConnectionSpeed(),
  });

  // Use sendBeacon for reliability during page unload
  if (navigator.sendBeacon) {
    navigator.sendBeacon(vitalsUrl, body);
  } else {
    fetch(vitalsUrl, {
      body,
      method: 'POST',
      keepalive: true,
    });
  }

  // Also track to analytics provider
  analytics.track('web_vital', {
    name: metric.name,
    value: metric.value,
    rating: metric.rating,
  });
}

export function reportWebVitals() {
  onCLS(sendToAnalytics);
  onFCP(sendToAnalytics);
  onFID(sendToAnalytics);
  onINP(sendToAnalytics);
  onLCP(sendToAnalytics);
  onTTFB(sendToAnalytics);
}

// app/layout.tsx
'use client';

import { useEffect } from 'react';
import { reportWebVitals } from '@/lib/vitals';

export function ClientLayout({ children }: { children: React.ReactNode }) {
  useEffect(() => {
    reportWebVitals();
  }, []);

  return <>{children}</>;
}
```

### Vitals Storage and Analysis

```typescript
// app/api/vitals/route.ts
import { NextRequest } from 'next/server';
import { db } from '@/lib/db';

interface WebVitalMetric {
  name: 'CLS' | 'FCP' | 'FID' | 'INP' | 'LCP' | 'TTFB';
  value: number;
  rating: 'good' | 'needs-improvement' | 'poor';
  page: string;
  speed: string;
  timestamp?: number;
}

export async function POST(req: NextRequest) {
  const metric: WebVitalMetric = await req.json();
  
  // Store in database for analysis
  await db.webVitals.create({
    data: {
      name: metric.name,
      value: metric.value,
      rating: metric.rating,
      page: metric.page,
      connectionSpeed: metric.speed,
      userAgent: req.headers.get('user-agent') || 'unknown',
      timestamp: new Date(metric.timestamp || Date.now()),
    },
  });

  // Alert on poor scores (optional)
  if (metric.rating === 'poor') {
    await alertOnPoorVital(metric);
  }

  return Response.json({ success: true });
}

async function alertOnPoorVital(metric: WebVitalMetric) {
  // Send to monitoring service (PagerDuty, Slack, etc.)
  if (metric.name === 'LCP' && metric.value > 4000) {
    await fetch(process.env.SLACK_WEBHOOK_URL!, {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({
        text: `🚨 Poor LCP detected: ${metric.value}ms on ${metric.page}`,
      }),
    });
  }
}

// Dashboard query
export async function GET(req: NextRequest) {
  const { searchParams } = new URL(req.url);
  const days = parseInt(searchParams.get('days') || '7');
  
  const startDate = new Date();
  startDate.setDate(startDate.getDate() - days);

  const stats = await db.webVitals.groupBy({
    by: ['name', 'rating'],
    where: {
      timestamp: { gte: startDate },
    },
    _count: {
      rating: true,
    },
    _avg: {
      value: true,
    },
  });

  return Response.json(stats);
}
```

## 33.3 Error Tracking and Monitoring

Implement comprehensive error tracking with Sentry or similar services.

### Sentry Integration

```typescript
// next.config.js
const { withSentryConfig } = require('@sentry/nextjs');

/** @type {import('next').NextConfig} */
const nextConfig = {
  // Your existing config
};

module.exports = withSentryConfig(nextConfig, {
  // Sentry webpack plugin options
  org: process.env.SENTRY_ORG,
  project: process.env.SENTRY_PROJECT,
  silent: !process.env.CI,
  
  // Upload source maps
  widenClientFileUpload: true,
  
  // Automatically tree-shake Sentry logger statements
  disableLogger: true,
  
  // Tunnel Sentry requests to avoid ad-blockers
  tunnelRoute: '/monitoring',
});
```

```typescript
// sentry.client.config.ts
import * as Sentry from '@sentry/nextjs';

Sentry.init({
  dsn: process.env.NEXT_PUBLIC_SENTRY_DSN,
  
  // Adjust sampling rate for production
  tracesSampleRate: process.env.NODE_ENV === 'production' ? 0.1 : 1.0,
  
  // Replay sampling
  replaysSessionSampleRate: 0.1,
  replaysOnErrorSampleRate: 1.0,
  
  integrations: [
    Sentry.replayIntegration({
      maskAllText: true,
      blockAllMedia: true,
    }),
    Sentry.feedbackIntegration({
      colorScheme: 'system',
    }),
  ],
  
  beforeSend(event) {
    // Sanitize PII before sending
    if (event.request?.headers) {
      delete event.request.headers['Authorization'];
      delete event.request.headers['Cookie'];
    }
    return event;
  },
});

// sentry.server.config.ts
import * as Sentry from '@sentry/nextjs';

Sentry.init({
  dsn: process.env.NEXT_PUBLIC_SENTRY_DSN,
  tracesSampleRate: 0.1,
  
  beforeSend(event) {
    // Filter out known errors
    if (event.exception?.values?.some(e => e.value?.includes('ResizeObserver'))) {
      return null;
    }
    return event;
  },
});
```

### Custom Error Boundary

```typescript
// components/error-boundary.tsx
'use client';

import * as Sentry from '@sentry/nextjs';
import { Component, ErrorInfo, ReactNode } from 'react';

interface Props {
  children: ReactNode;
  fallback?: ReactNode;
}

interface State {
  hasError: boolean;
  errorId?: string;
}

export class ErrorBoundary extends Component<Props, State> {
  public state: State = {
    hasError: false,
  };

  public static getDerivedStateFromError(): State {
    return { hasError: true };
  }

  public componentDidCatch(error: Error, errorInfo: ErrorInfo) {
    // Log to Sentry with context
    Sentry.withScope((scope) => {
      scope.setExtras(errorInfo);
      const eventId = Sentry.captureException(error);
      this.setState({ errorId: eventId });
    });
  }

  public render() {
    if (this.state.hasError) {
      return this.props.fallback || (
        <div className="p-8 text-center">
          <h2 className="text-2xl font-bold text-red-600 mb-4">
            Something went wrong
          </h2>
          <p className="text-gray-600 mb-4">
            Our team has been notified. Please try refreshing the page.
          </p>
          {this.state.errorId && (
            <p className="text-sm text-gray-400">
              Error ID: {this.state.errorId}
            </p>
          )}
          <button
            onClick={() => window.location.reload()}
            className="mt-4 px-4 py-2 bg-blue-600 text-white rounded"
          >
            Refresh Page
          </button>
        </div>
      );
    }

    return this.props.children;
  }
}
```

## 33.4 Custom Event Tracking

Track user interactions and business metrics throughout your application.

### Event Tracking Hook

```typescript
// hooks/use-analytics.ts
'use client';

import { useCallback } from 'react';
import { analytics } from '@/lib/analytics/provider';

type EventName = 
  | 'page_view'
  | 'click'
  | 'form_submit'
  | 'purchase'
  | 'signup'
  | 'feature_used'
  | 'error';

interface TrackEvent {
  name: EventName;
  properties?: Record<string, any>;
}

export function useAnalytics() {
  const track = useCallback((event: TrackEvent) => {
    analytics.track(event.name, {
      ...event.properties,
      url: window.location.href,
      referrer: document.referrer,
    });
  }, []);

  const trackClick = useCallback((elementName: string, metadata?: Record<string, any>) => {
    track({
      name: 'click',
      properties: {
        element: elementName,
        ...metadata,
      },
    });
  }, [track]);

  const trackFormSubmit = useCallback((formName: string, success: boolean, metadata?: Record<string, any>) => {
    track({
      name: 'form_submit',
      properties: {
        form: formName,
        success,
        ...metadata,
      },
    });
  }, [track]);

  return {
    track,
    trackClick,
    trackFormSubmit,
  };
}

// Server-side tracking utility
// lib/analytics/server.ts
import { headers } from 'next/headers';

export async function trackServerEvent(
  eventName: string,
  properties: Record<string, any>
) {
  const headersList = headers();
  const userAgent = headersList.get('user-agent') || 'unknown';
  const ip = headersList.get('x-forwarded-for') || 'unknown';
  
  // Hash IP for privacy
  const hashedIp = await hashString(ip);
  
  await fetch(process.env.ANALYTICS_ENDPOINT!, {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({
      event: eventName,
      properties,
      metadata: {
        userAgent,
        ip: hashedIp,
        timestamp: new Date().toISOString(),
      },
    }),
  });
}

async function hashString(str: string): Promise<string> {
  const encoder = new TextEncoder();
  const data = encoder.encode(str);
  const hashBuffer = await crypto.subtle.digest('SHA-256', data);
  const hashArray = Array.from(new Uint8Array(hashBuffer));
  return hashArray.map(b => b.toString(16).padStart(2, '0')).join('');
}
```

## 33.5 Real User Monitoring (RUM)

Implement session recording and performance monitoring for real debugging insights.

### Session Replay with Privacy Controls

```typescript
// components/privacy-controls.tsx
'use client';

import { useEffect, useState } from 'react';

export function PrivacyControls() {
  const [consent, setConsent] = useState<boolean | null>(null);

  useEffect(() => {
    const stored = localStorage.getItem('analytics-consent');
    if (stored) setConsent(stored === 'granted');
  }, []);

  const handleConsent = (granted: boolean) => {
    localStorage.setItem('analytics-consent', granted ? 'granted' : 'denied');
    setConsent(granted);
    
    if (granted) {
      // Initialize Sentry Replay
      import('@sentry/nextjs').then(Sentry => {
        Sentry.init({
          replaysSessionSampleRate: 0.1,
        });
      });
    }
  };

  if (consent !== null) return null;

  return (
    <div className="fixed bottom-4 right-4 p-4 bg-white rounded-lg shadow-lg border max-w-sm z-50">
      <p className="text-sm text-gray-700 mb-3">
        We collect anonymous usage data to improve your experience. 
        No personal information is stored.
      </p>
      <div className="flex gap-2">
        <button
          onClick={() => handleConsent(false)}
          className="px-3 py-1.5 text-sm border rounded hover:bg-gray-50"
        >
          Decline
        </button>
        <button
          onClick={() => handleConsent(true)}
          className="px-3 py-1.5 text-sm bg-blue-600 text-white rounded hover:bg-blue-700"
        >
          Allow
        </button>
      </div>
    </div>
  );
}
```

### Performance Observer API

```typescript
// hooks/use-performance-monitoring.ts
'use client';

import { useEffect } from 'react';

export function usePerformanceMonitoring() {
  useEffect(() => {
    // Observe long tasks
    if ('PerformanceObserver' in window) {
      const observer = new PerformanceObserver((list) => {
        for (const entry of list.getEntries()) {
          // Report long tasks (> 50ms) that block main thread
          if (entry.duration > 50) {
            fetch('/api/performance/long-task', {
              method: 'POST',
              body: JSON.stringify({
                duration: entry.duration,
                startTime: entry.startTime,
                url: window.location.pathname,
              }),
              keepalive: true,
            });
          }
        }
      });
      
      observer.observe({ entryTypes: ['longtask'] });
      
      return () => observer.disconnect();
    }
  }, []);

  useEffect(() => {
    // Monitor resource loading
    const observer = new PerformanceObserver((list) => {
      const entries = list.getEntries();
      
      // Find slow resources
      const slowResources = entries.filter((entry: any) => 
        entry.responseEnd - entry.startTime > 1000
      );
      
      if (slowResources.length > 0) {
        fetch('/api/performance/resources', {
          method: 'POST',
          body: JSON.stringify({
            resources: slowResources.map((r: any) => ({
              name: r.name,
              duration: r.responseEnd - r.startTime,
              size: r.transferSize,
            })),
          }),
          keepalive: true,
        });
      }
    });
    
    observer.observe({ entryTypes: ['resource'] });
    
    return () => observer.disconnect();
  }, []);
}
```

## Key Takeaways from Chapter 33

1. **Privacy-First Analytics**: Use cookie-less tracking solutions like Vercel Analytics or implement custom solutions that respect `navigator.doNotTrack` and GDPR consent requirements. Hash IP addresses and avoid collecting personally identifiable information (PII) in your analytics pipeline.

2. **Core Web Vitals**: Instrument the `web-vitals` library to capture real-user metrics (CLS, LCP, INP) in production. Use `navigator.sendBeacon()` for reliable transmission during page unload, and store metrics in time-series databases for trend analysis and alerting on performance regressions.

3. **Error Tracking**: Integrate Sentry or similar services with source map uploading for production debugging. Configure sampling rates to balance data collection with cost, and implement `beforeSend` hooks to sanitize sensitive data like Authorization headers and user emails before transmission.

4. **Event Tracking**: Build a unified tracking layer that works across server components (using `headers()` for request metadata) and client components (using context or global state). Implement batching and queueing to minimize network overhead while ensuring critical events (purchases, signups) are sent immediately.

5. **RUM and Session Replay**: Implement privacy controls for session recording, masking sensitive input fields and offering opt-out mechanisms. Use the Performance Observer API to detect long tasks and resource loading issues that impact user experience but may not appear in synthetic testing.

6. **Alerting**: Set up threshold-based alerts for poor Core Web Vitals scores (LCP > 4s, CLS > 0.25) and error rate spikes. Use webhook integrations to notify teams via Slack or PagerDuty when user experience degrades beyond acceptable limits.

7. **Data Retention**: Implement data retention policies compliant with privacy regulations—automatically purge raw analytics data after 30-90 days while keeping aggregated statistics for long-term trend analysis.

## Coming Up Next

**Chapter 34: Content Management**

With comprehensive monitoring in place, you can now focus on managing the content that drives your application. In Chapter 34, we'll explore headless CMS integration patterns, MDX for content-driven pages, real-time preview modes, content caching strategies, and multi-tenant content architectures. You'll learn how to structure your Next.js application for dynamic content updates without code deployments, enabling marketing teams and content creators to iterate rapidly.